# Content Aggregator Database Explorer

Explore the content database with SQL queries and visualizations.

In [ ]:
import os
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv

load_dotenv()

# Database connection
def get_connection():
    return psycopg2.connect(
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT"),
        database=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD")
    )

def query(sql):
    """Execute SQL and return as DataFrame."""
    conn = get_connection()
    df = pd.read_sql_query(sql, conn)
    conn.close()
    return df

## Quick Stats

In [ ]:
# Total content count
query("SELECT COUNT(*) as total FROM content")

In [ ]:
# Content by source
query("""
    SELECT source_name, COUNT(*) as count
    FROM content
    GROUP BY source_name
    ORDER BY count DESC
""")

In [ ]:
# Watched vs unwatched
query("""
    SELECT 
        consumed,
        COUNT(*) as count
    FROM content
    GROUP BY consumed
""")

## Recent Videos

In [ ]:
# Last 10 collected videos
query("""
    SELECT 
        title,
        source_name,
        published_at,
        estimated_duration / 60 as duration_min,
        consumed
    FROM content
    ORDER BY collected_at DESC
    LIMIT 10
""")

## Duration Analysis

In [ ]:
# Duration statistics
query("""
    SELECT 
        MIN(estimated_duration / 60) as min_minutes,
        MAX(estimated_duration / 60) as max_minutes,
        AVG(estimated_duration / 60) as avg_minutes
    FROM content
    WHERE estimated_duration IS NOT NULL
""")

In [ ]:
# Duration distribution by channel
df = query("""
    SELECT 
        source_name,
        AVG(estimated_duration / 60) as avg_duration_min
    FROM content
    WHERE estimated_duration IS NOT NULL
    GROUP BY source_name
    ORDER BY avg_duration_min DESC
""")

plt.figure(figsize=(10, 6))
plt.barh(df['source_name'], df['avg_duration_min'])
plt.xlabel('Average Duration (minutes)')
plt.title('Average Video Duration by Channel')
plt.tight_layout()
plt.show()

## Custom Queries

Use the cell below to run your own SQL queries:

In [ ]:
# Your custom query here
query("""
    SELECT *
    FROM content
    LIMIT 5
""")

## Full Table Schema

In [ ]:
# View table structure
query("""
    SELECT 
        column_name,
        data_type,
        is_nullable
    FROM information_schema.columns
    WHERE table_name = 'content'
    ORDER BY ordinal_position
""")